In [2]:
from env_loader import load_env

repositoryDir= "./repository"
docs= [ ]

# Load OpenAI and ActiveLoop tokens into environment
load_env("./")

In [3]:
import os
from langchain.document_loaders import TextLoader

# Load all files inside the repository
for dirpath, dirnames, filenames in os.walk(repositoryDir):
  for file in filenames:
    try:
      loader= TextLoader(os.path.join(dirpath, file), encoding= "utf-8")
      docs.extend(loader.load_and_split( ))
    except Exception as e:
      pass

In [5]:
from langchain.text_splitter import CharacterTextSplitter

# Chunk the files
texts = CharacterTextSplitter(chunk_size= 1000, chunk_overlap= 0).split_documents(docs)

Created a chunk of size 1020, which is longer than the specified 1000
Created a chunk of size 1540, which is longer than the specified 1000
Created a chunk of size 1083, which is longer than the specified 1000
Created a chunk of size 1676, which is longer than the specified 1000
Created a chunk of size 1162, which is longer than the specified 1000


In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

try:
  # Create DeepLake dataset
  global db
  db= DeepLake(
    dataset_path= "hub://archis/nicobraun-kobold",
    embedding_function= OpenAIEmbeddings(disallowed_special=( )),
    public= True
  )
except:
  db= DeepLake(
    dataset_path= "hub://archis/nicobraun-kobold",
    embedding_function= OpenAIEmbeddings(disallowed_special=( )),
  )

|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/archis/nicobraun-kobold



\

hub://archis/nicobraun-kobold loaded successfully.

Deep Lake Dataset in hub://archis/nicobraun-kobold already exists, loading from the storage
Dataset(path='hub://archis/nicobraun-kobold', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype    shape    dtype  compression
  -------   -------  -------  -------  ------- 
 embedding  generic   (0,)    float32   None   
    ids      text     (0,)      str     None   
 metadata    json     (0,)      str     None   
   text      text     (0,)      str     None   


In [7]:
# Create embeddings and upload to ActiveLoop
db.add_documents(texts)

Evaluating ingest: 100%|██████████| 1/1 [00:30<00:00
-

Dataset(path='hub://archis/nicobraun-kobold', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape      dtype  compression
  -------   -------    -------    -------  ------- 
 embedding  generic  (289, 1536)  float32   None   
    ids      text     (289, 1)      str     None   
 metadata    json     (289, 1)      str     None   
   text      text     (289, 1)      str     None   


['aca71a1a-0bc2-11ee-85ba-41863261828c',
 'aca71a1b-0bc2-11ee-85ba-41863261828c',
 'aca71a1c-0bc2-11ee-85ba-41863261828c',
 'aca71a1d-0bc2-11ee-85ba-41863261828c',
 'aca71a1e-0bc2-11ee-85ba-41863261828c',
 'aca71a1f-0bc2-11ee-85ba-41863261828c',
 'aca71a20-0bc2-11ee-85ba-41863261828c',
 'aca71a21-0bc2-11ee-85ba-41863261828c',
 'aca71a22-0bc2-11ee-85ba-41863261828c',
 'aca71a23-0bc2-11ee-85ba-41863261828c',
 'aca71a24-0bc2-11ee-85ba-41863261828c',
 'aca71a25-0bc2-11ee-85ba-41863261828c',
 'aca71a26-0bc2-11ee-85ba-41863261828c',
 'aca71a27-0bc2-11ee-85ba-41863261828c',
 'aca71a28-0bc2-11ee-85ba-41863261828c',
 'aca71a29-0bc2-11ee-85ba-41863261828c',
 'aca71a2a-0bc2-11ee-85ba-41863261828c',
 'aca71a2b-0bc2-11ee-85ba-41863261828c',
 'aca71a2c-0bc2-11ee-85ba-41863261828c',
 'aca71a2d-0bc2-11ee-85ba-41863261828c',
 'aca71a2e-0bc2-11ee-85ba-41863261828c',
 'aca71a2f-0bc2-11ee-85ba-41863261828c',
 'aca71a30-0bc2-11ee-85ba-41863261828c',
 'aca71a31-0bc2-11ee-85ba-41863261828c',
 'aca71a32-0bc2-

In [9]:
# Construct the retriever, then construct the Conversational Chain
retriever = db.as_retriever( )
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 10

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name= "gpt-3.5-turbo")
qa = ConversationalRetrievalChain.from_llm(model, retriever= retriever)

In [14]:
chatHistory= [ ]

def chat(questions):
  for question in questions:
    result = qa({ "question": question, "chat_history": chatHistory })

    chatHistory.append((question, result["answer"]))

    print(f"=> Question |: {question} \n")
    print(f"Answer |: {result['answer']} \n")

In [16]:
chat([
  """
    Explain "func (g generator) Generate(conf *kobold.NormalizedConfig) (http.Handler, error) {"
    in /internal/server/mux.go in Kobold repository. 
  """
])

=> Question |: 
    Explain "func (g generator) Generate(conf *kobold.NormalizedConfig) (http.Handler, error) {"
    in /internal/server/mux.go in Kobold repository. 
   

Answer |: The "func (g generator) Generate(conf *kobold.NormalizedConfig) (http.Handler, error)" function in the "mux.go" file located in the "internal/server" directory of the Kobold repository is used to generate an HTTP request handler based on a given configuration. The function initializes repositories, sets up channels for subscriptions, and creates endpoints that are used to handle webhook events. It returns a handler that will be used by the server to handle incoming requests. 

